### Question: Predicting # of Flight delays given certain weather and past flight data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Read in csv files from Kaggle
pressure_df = pd.read_csv('weather_data/pressure.csv')
humidity_df = pd.read_csv('weather_data/humidity.csv')
temperature_df = pd.read_csv('weather_data/temperature.csv')
wind_direction_df = pd.read_csv('weather_data/wind_direction.csv')
wind_speed_df = pd.read_csv('weather_data/wind_speed.csv')
weather_description_df = pd.read_csv("weather_data/weather_description.csv")


In [ ]:
# Top 5 cities with most values in flight delay dataset
cities = ["Atlanta",'Chicago','Dallas','Charlotte','Denver']

In [ ]:
def clean_df(dataframe,name):
    dataframe = dataframe[["datetime"] + cities] # call subset of df
    dataframe_2015_index = list(dataframe.index[dataframe['datetime'] == "2015-01-01 00:00:00"])
    # going from 2015 - 2017
    dataframe = dataframe.iloc[dataframe_2015_index[0]:,:]
    # add name to columns specifying originating df. Need this specified when joining table
    new_names = dict(zip(list(cities),list(np.core.defchararray.add(cities,name))))
    new_names['datetime'] = 'datetime'
    dataframe = dataframe.rename(columns = new_names)
    return dataframe
        

In [ ]:
# New cleaned and correctly labeled df
pressure_df = clean_df(pressure_df, '_pressure')
humidity_df = clean_df(humidity_df, '_humidity')
temperature_df = clean_df(temperature_df, '_temperature')
wind_direction_df = clean_df(wind_direction_df, '_wind_direction')
wind_speed_df = clean_df(wind_speed_df, '_wind_speed')
weather_description_df = clean_df(weather_description_df, '_description')

In [ ]:
# Merge the individual datasets on datetime
A = pd.merge(pressure_df.merge(humidity_df, how = 'outer'),temperature_df, how = 'outer')
B = pd.merge(wind_direction_df.merge(wind_speed_df, how = 'outer'), weather_description_df, how = 'outer')
weather_df = A.merge(B, how = 'outer')
    

In [ ]:
weather_df.head()

In [ ]:
# Check for column names that have null values
null_list = list(weather_df.columns[weather_df.isna().any()])
for column in null_list:
    # check missing values ratio to dataset size
    print(column, (weather_df[column].isna().sum()/weather_df.shape[0])*100)
    
# Output:
# Chicago_pressure 0.003915886752555116
# Atlanta_humidity 0.07440184829854721
# Chicago_humidity 0.003915886752555116
# Dallas_humidity 0.12922426283431884
# Charlotte_humidity 0.775345577005913
# Denver_humidity 0.01174766025766535
# Dallas_wind_speed 0.003915886752555116
# Charlotte_wind_speed 0.003915886752555116

# Since they're so minute, will just drop rows that have null values

In [ ]:
weather_df.shape # 25537 rows
weather_df = weather_df.dropna()
weather_df.shape # 240 rows dropped

In [ ]:
weather_df.head()

In [ ]:
# Split datetime column into Years, Months, Days and time column
Year = []
Month = []
Day = []
Time = []
for datetime in weather_df['datetime']:
    date, time = datetime.split(' ')
    year, month, day = date.split('-')
    Year.append(year)
    Month.append(month)
    Day.append(day)
    Time.append(time)

# Convert arrays to pandas Series and make it int type, so when merging, the object-type will be same
weather_df['Year'] = pd.Series(Year, dtype='int32')
weather_df["Month"] = pd.Series(Month, dtype='int32')
weather_df['Day'] = pd.Series(Day, dtype='int32')
weather_df['Time'] = Time

In [ ]:
# Rearranging them so date/time is in beginning
weather_df = weather_df[['Year', 'Month', 'Day','Time','Atlanta_pressure', 'Chicago_pressure', 'Dallas_pressure',\
       'Charlotte_pressure', 'Denver_pressure', 'Atlanta_humidity',\
       'Chicago_humidity', 'Dallas_humidity', 'Charlotte_humidity',\
       'Denver_humidity', 'Atlanta_temperature', 'Chicago_temperature',\
       'Dallas_temperature', 'Charlotte_temperature', 'Denver_temperature',\
       'Atlanta_wind_direction', 'Chicago_wind_direction',\
       'Dallas_wind_direction', 'Charlotte_wind_direction',\
       'Denver_wind_direction', 'Atlanta_wind_speed', 'Chicago_wind_speed',\
       'Dallas_wind_speed', 'Charlotte_wind_speed', 'Denver_wind_speed',\
       'Atlanta_description', 'Chicago_description', 'Dallas_description',\
       'Charlotte_description', 'Denver_description']]
weather_df.head()